<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/QAS_BERT_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os 
import sys 
import json
from platform import python_version

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,MultiHeadAttention

import tensorflow_hub as hub
!pip install -q tensorflow-text

import tensorflow_text as text

In [2]:
print('Python Version : ' ,python_version())
print('tensorflow version : ', tf.__version__)
print('pandas version : ', pd.__version__)
print('numpy version : ', np.__version__)
print('tf_hub version : ', hub.__version__)


Python Version :  3.7.10
tensorflow version :  2.4.1
pandas version :  1.1.5
numpy version :  1.19.5
tf_hub version :  0.11.0


In [3]:
train_path = keras.utils.get_file("train.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")
eval_path = keras.utils.get_file("eval.json", "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json")
with open(train_path) as f: raw_train_data = json.load(f)
with open(eval_path) as f: raw_eval_data = json.load(f)
# max_seq_length = 384

In [4]:
print(raw_train_data.keys())
print(raw_train_data['data'][0].keys())
print(raw_train_data['data'][0]['title'][0:])
print(raw_train_data['data'][1]['title'][0:])

dict_keys(['data', 'version'])
dict_keys(['title', 'paragraphs'])
University_of_Notre_Dame
Beyoncé


In [5]:
print(raw_train_data['data'][3]['paragraphs'][0].keys())

dict_keys(['context', 'qas'])


In [6]:
print(raw_train_data['data'][0]['paragraphs'][0]['qas'])

[{'answers': [{'answer_start': 515, 'text': 'Saint Bernadette Soubirous'}], 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'id': '5733be284776f41900661182'}, {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ'}], 'question': 'What is in front of the Notre Dame Main Building?', 'id': '5733be284776f4190066117f'}, {'answers': [{'answer_start': 279, 'text': 'the Main Building'}], 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?', 'id': '5733be284776f41900661180'}, {'answers': [{'answer_start': 381, 'text': 'a Marian place of prayer and reflection'}], 'question': 'What is the Grotto at Notre Dame?', 'id': '5733be284776f41900661181'}, {'answers': [{'answer_start': 92, 'text': 'a golden statue of the Virgin Mary'}], 'question': 'What sits on top of the Main Building at Notre Dame?', 'id': '5733be284776f4190066117e'}]


In [23]:
def text_maker (data):
  squad_examples = []
  full_context = []
  full_question = []
  full_answer = []
  for item in raw_train_data["data"]:
    # full_context = []
      for para in item["paragraphs"]:
          context = para["context"]
          full_context.append(context)
          for qa in para["qas"]:
              question = qa["question"]
              full_question.append(question)
              if "answers" in qa:
                  answer_text = qa["answers"][0]["text"]
                  full_answer.append(answer_text)
                  all_answers = [_["text"] for _ in qa["answers"]]
                  start_char_idx = qa["answers"][0]["answer_start"]
                # squad_eg = Sample(question, context, start_char_idx, answer_text, all_answers)
            # else:
                # squad_eg = Sample(question, context)
            # squad_eg.preprocess()
            # squad_examples.append(squad_eg)

  return(full_context, full_question, full_answer)

In [24]:
train_context, train_question,train_answer= text_maker(raw_train_data)
test_context, test_question, test_answer = text_maker(raw_eval_data)